In [1]:
# Reading from a MCAP file
# sudo apt install ros-$ROS_DISTRO-ros-base ros-$ROS_DISTRO-ros2bag ros-$ROS_DISTRO-rosbag2-transport ros-$ROS_DISTRO-rosbag2-storage-mcap 
from mcap import __version__
print(__version__)


"""script that reads ROS2 messages from an MCAP bag using the rosbag2_py API."""
import argparse
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message
import rosbag2_py


def read_messages(input_bag: str):
    reader = rosbag2_py.SequentialReader()
    reader.open(
        rosbag2_py.StorageOptions(uri=input_bag, storage_id="mcap"),
        rosbag2_py.ConverterOptions(
            input_serialization_format="cdr", output_serialization_format="cdr"
        ),
    )

    topic_types = reader.get_all_topics_and_types()

    def typename(topic_name):
        for topic_type in topic_types:
            if topic_type.name == topic_name:
                return topic_type.type
        raise ValueError(f"topic {topic_name} not in bag")

    while reader.has_next():
        topic, data, timestamp = reader.read_next()
        msg_type = get_message(typename(topic))
        msg = deserialize_message(data, msg_type)
        yield topic, msg, timestamp
    del reader


1.1.1


```
Topic: /lexus3/vehicle_steering | Type: std_msgs/msg/Float32 | Count: 4400
Topic: /lexus3/vehicle_speed | Type: std_msgs/msg/Float32 | Count: 4400
Topic: /lexus3/gps/duro/current_pose | Type: geometry_msgs/msg/PoseStamped | Count: 2970
Topic: /lexus3/gps/duro/status_string | Type: std_msgs/msg/String | Count: 2970
Topic: /lexus3/gps/duro/imu | Type: sensor_msgs/msg/Imu | Count: 14701
Topic: /lexus3/gps/duro/mag | Type: sensor_msgs/msg/MagneticField | Count: 1837
Topic: /tf_static | Type: tf2_msgs/msg/TFMessage | Count: 10
Topic: /tf | Type: tf2_msgs/msg/TFMessage | Count: 2970

Topic: /nissan/gps/duro/current_pose | Type: geometry_msgs/msg/PoseStamped | Count: 1216
Topic: /nissan/gps/duro/imu | Type: sensor_msgs/msg/Imu | Count: 6021
Topic: /nissan/gps/duro/mag | Type: sensor_msgs/msg/MagneticField | Count: 1505
Topic: /nissan/gps/duro/status_string | Type: std_msgs/msg/String | Count: 1216
Topic: /nissan/vehicle_speed | Type: std_msgs/msg/Float32 | Count: 1823
Topic: /nissan/vehicle_steering | Type: std_msgs/msg/Float32 | Count: 1823
```

In [2]:
inpu = "/mnt/c/bag/jkkds02/nissan_zala_90_mixed_0.mcap"

reader = rosbag2_py.SequentialReader()
reader.open(rosbag2_py.StorageOptions(uri=inpu, storage_id="mcap"), rosbag2_py.ConverterOptions(input_serialization_format="cdr", output_serialization_format="cdr"),)


# list topics
topics = reader.get_all_topics_and_types()

for topic in topics:
    print(topic.name)


/nissan/gps/duro/status_string
/nissan/gps/duro/mag
/nissan/gps/duro/imu
/nissan/gps/duro/current_pose
/nissan/vehicle_speed
/nissan/vehicle_steering


In [3]:
import pandas as pd

data = []
for topic in topics:
    data.append([topic.name, topic.type])

data_df = pd.DataFrame(data)
data_df.columns = ['Topic', 'Type']
data_df

,Topic,Type
0,/nissan/gps/duro/status_string,std_msgs/msg/String
1,/nissan/gps/duro/mag,sensor_msgs/msg/MagneticField
2,/nissan/gps/duro/imu,sensor_msgs/msg/Imu
3,/nissan/gps/duro/current_pose,geometry_msgs/msg/PoseStamped
4,/nissan/vehicle_speed,std_msgs/msg/Float32
5,/nissan/vehicle_steering,std_msgs/msg/Float32
